In [1]:
import numpy as np
np.random.seed(1)

In [2]:
def softmax(x_):
    x = np.atleast_2d(x_) #turns it from 1d vector to 2d
    temp = np.exp(x)
    return temp / np.sum(temp, axis =1, keepdims = True) #axis = 1 is sum the columns

In [3]:
word_vects = {}
word_vect_template  = np.array([[0.,0.,0.]])

word_list = ['yankees','bears','braves','red','sox','lose','defeat','beat','tie']

for w in word_list:
    word_vects[w] = word_vect_template

In [4]:
sent2output = np.random.rand(3, len(word_vects)) #[3,9]
identity = np.eye(3)

In [5]:
alpha = 0.1
y = np.array([1,0,0,0,0,0,0,0,0])

for i in range(100):
    
    layer_0 = word_vects['red']
    layer_1 = layer_0.dot(identity) + word_vects['sox']
    layer_2 = layer_1.dot(identity) + word_vects['defeat']
    
    pred = softmax(layer_2.dot(sent2output))
    
    pred_delta = pred - y

    layer_2_delta = pred_delta.dot(sent2output.T)
    defeat_delta = layer_2_delta*1

    layer_1_delta = layer_2_delta.dot(identity.T)
    sox_delta = layer_1_delta*1

    layer_0_delta = layer_1_delta.dot(identity.T)

    word_vects['red'] -= layer_0_delta*alpha
    word_vects['sox'] -= sox_delta*alpha
    word_vects['defeat'] -= defeat_delta*alpha

    identity -= np.outer(layer_1_delta, layer_0) * alpha
    identity -= np.outer(layer_2_delta, layer_1) * alpha

    sent2output -= np.outer(layer_2, pred_delta) * alpha



In [6]:
print(f"Pred:{pred}")

Pred:[[9.99115839e-01 3.24142757e-04 9.89827279e-06 4.58200097e-06
  1.55786539e-04 1.48957833e-05 8.48400706e-06 8.53677564e-06
  3.57835314e-04]]


# What is normalize

In [121]:
w01 = np.random.rand(3,4)

In [122]:
(w01 * w01) #element wise multiplication 

array([[3.95219721e-04, 6.87015833e-04, 8.01257264e-04, 6.06198898e-02],
       [7.39648073e-01, 2.90338916e-01, 3.05612140e-01, 7.09016024e-01],
       [1.54190122e-02, 7.79435266e-02, 3.43113924e-01, 9.40115915e-01]])

In [123]:
norms = np.sum(w01*w01, axis = 1) #[3,1]
norms.resize(norms.shape[0],1)

normed_weights = w01 * norms #ew multiplation/scaled by the column sum ; give you 0-1 for each column 

# RNN with dictionary

In [124]:
# With dictonionary 

tokens = [["Mary","had","a","little","lamb"],
         ["Its","fleec","was","white","as","snow"],
          ["And","everywhere","that","Mary","went"],
          ["The","lamb","was","sure","to","go"]]

In [127]:
vocab = set()
wrdcnt = 0
for sent in tokens:
    for word in sent:
        vocab.add(word)
        wrdcnt += 1

vocab = list(vocab)
print(len(vocab))
print(wrdcnt)

19
22


In [135]:
word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i

In [141]:
vocab

['went',
 'to',
 'And',
 'had',
 'a',
 'was',
 'everywhere',
 'as',
 'Mary',
 'snow',
 'Its',
 'that',
 'go',
 'little',
 'white',
 'sure',
 'The',
 'fleec',
 'lamb']

In [131]:
def word2indice(sentence):
    idx = list()
    for word in sentence:
        idx.append(word2index[word])
    return idx 

In [134]:
embed_size = 10

embed = (np.random.rand(len(vocab), embed_size) - 0.5)*0.1
recurrent = np.eye(embed_size)
start = np.zeros(embed_size)
decoder = (np.random.rand(embed_size, len(vocab)) - 0.5) * 0.1
one_hot = np.eye(len(vocab))

In [157]:
def predict(sent):
    
    layers = list()
    layer = {}
    layer['hidden'] = start
    layers.append(layer)
    
    loss = 0
    
    preds = list()
    
    for target_i in range(len(sent)):
        
        layer = {}
        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder))
        print(layer['pred'].shape)
        
        loss += -np.log(layer['pred'],[sent[target_i]])
        
        layer['hidden'] = layers[-1]['hidden'].dot(recurrent) + embed[sent[target_i]]
        
        layers.append(layer)
    return layers, loss

In [158]:
predict(tokens[3])

(1, 19)


TypeError: return arrays must be of ArrayType

In [151]:
ti = 3
sent[ti]

'sure'

In [153]:
embed.shape

(19, 10)